### Scraping Reddit data

## Install Required Libraries

In [ ]:
# Install required libraries
!pip install praw pandas

## Import Libraries

In [ ]:
# Import libraries
import praw
import pandas as pd
from datetime import datetime

## Set Up Reddit API Credentials

In [ ]:

# API credentials
reddit = praw.Reddit(
    client_id='eGcM2W6mG3BbMXJveFmxIQ',
    client_secret='jVrEOLaxR2gPr3QIsRhf4hUsTMOkvA',
    user_agent='sentiment-project-script'
)

## Define Keywords and Subreddits

In [ ]:
# Define the keywords to search
keywords = ['foodpanda', 'grabfood', 'shopeefood', 'grab food', 'food panda', 'grab delivery', 'shopee food']

# Use only known valid subreddits
subreddits = ['malaysia', 'malaysianfood', 'food', 'casualmalaysia']

## Initialize Data Storage

In [ ]:
# Prepare to store results with comments included
posts_data = []

## Scrape Reddit Including Comments

In [ ]:
# Scrape Reddit including comments
for subreddit_name in subreddits:
    subreddit = reddit.subreddit(subreddit_name)
    for keyword in keywords:
        print(f"🔍 Searching for '{keyword}' in r/{subreddit_name}")
        try:
            for post in subreddit.search(keyword, sort='new', limit=100):
                post_date = datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d')
                post.comments.replace_more(limit=0)  # Flatten the comment tree
                comments = [comment.body for comment in post.comments.list()[:10]]  # Get top 10 comments only

                posts_data.append({
                    'title': post.title,
                    'text': post.selftext,
                    'comments': " || ".join(comments),  # Combine comments with separator
                    'date': post_date,
                    'score': post.score,
                    'num_comments': post.num_comments,
                    'created_utc': post.created_utc,
                    'subreddit': subreddit_name,
                    'keyword': keyword,
                    'url': post.url
                })
        except Exception as e:
            print(f"⚠️ Error in r/{subreddit_name}: {e}")


## Convert Data to DataFrame

In [ ]:

# Convert to DataFrame
df = pd.DataFrame(posts_data)

## Save Data to CSV


In [ ]:
# Save to CSV
df.to_csv('reddit_food_delivery_sentiments4.csv', index=False)


## Show Preview of the Data

In [ ]:
# Show preview
df.head()
